In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Conv1D, AveragePooling1D, UpSampling1D

import numpy as np
import pandas as pd

from datetime import datetime
import os
import sys
import pickle
import time
import argparse
from tqdm import tqdm

sys.path.insert(0, '/vast/nj594/xai/helpers')
from evaluate import evaluate

# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [2]:
# Load Data

data_dir = '../data'

X_val = np.load(os.path.join(data_dir, 'X_val.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test.npy'), allow_pickle=True)

y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)

preds = np.load(os.path.join(data_dir, 'predictions.npy'), allow_pickle=True)
preds_discrete = np.eye(2)[preds.argmax(1)]

preds_val = np.load(os.path.join(data_dir, 'predictions_val.npy'), allow_pickle=True)
preds_discrete_val = np.eye(2)[preds_val.argmax(1)]

In [3]:
# Load Evaluator Model

eval_dir = os.path.join(os.getcwd(), 'evaluator-data')
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

/ext3/miniconda3/lib/python3.7/site-packages/keras/layers/core.py:1045: UserWarning: network is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)
2022-05-05 13:36:40.434730: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 13:36:40.435245: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32252 MB memory:  -> device: 0, name: Vega 20, pci bus id: 0000:c5:00.0


In [4]:
methods = ['fastshap', 'fastshap-data', 'kernelshap', 'smoothgrad', 
           'integratedgradients', 'kernelshap_s', 'kernelshap_s-data', 'realx', 'realx-data']
results = {}
for method in ['fastshap-dkl', 'kernelshap_s-dkl']:
    print(method)
    results[method] = {} 
    ## Load Explanations
    if method  not in ['smoothgrad', 'integratedgradients', 'realx', 'realx-data', 'gradcam']:
        exp_file_retro = os.path.join('..', method, 'shap_values-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'shap_values-prospective.pkl')
    elif method == 'gradcam':
        exp_file_retro = os.path.join('..', method, 'explanations.pkl')
        exp_file_pro = exp_file_retro
    else:
        exp_file_retro = os.path.join('..', method, 'explanations-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'explanations-prospective.pkl')
        
    with open(exp_file_retro, 'rb') as f:
        explanations_retro = pickle.load(f)
    with open(exp_file_pro, 'rb') as f:
        explanations_pro = pickle.load(f)
        
    ## Evaluate Explanations
    
    ### Retrospective Evaluation

    # Exclusion
    results[method]['retro_ex'] = evaluate(X_test, explanations_retro, evaluator_model, y_test, y_test, 
                                           mode = 'exclude', method = method)
    # Inclusion
    results[method]['retro_in'] = evaluate(X_test, explanations_retro, evaluator_model, y_test, y_test, 
                                           mode = 'include', method = method)
    
    ### Prospective Evaluation 

    # Exclusion
    results[method]['pro_ex'] = evaluate(X_test, explanations_pro, evaluator_model, preds_discrete, y_test, 
                                         mode = 'exclude', method = method)
    # Inclusion
    results[method]['pro_in'] = evaluate(X_test, explanations_pro, evaluator_model, preds_discrete, y_test, 
                                         mode = 'include', method = method) 
    print(results)
    
    with open('results-{}.pkl'.format(method), 'wb') as f:
        pickle.dump(results[method], f)

fastshap-dkl
100


2022-05-05 13:36:43.294905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-05 13:36:43.313424: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:36:43.317129: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:36:43.319353: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:36:43.937786: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


99


2022-05-05 13:36:51.801349: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:36:51.805055: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:36:51.807289: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


95


...

2022-05-05 13:38:23.453380: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:38:23.457022: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:38:23.459240: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


0


2022-05-05 13:38:24.347097: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:38:24.350919: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-05 13:38:24.353145: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


{'fastshap-dkl': {'retro_ex': {'acc': {100: 0.9232547387887193, 99: 0.9232547387887193, 95: 0.9195561719833565, 90: 0.919093851132686, 85: 0.917706888580675, 75: 0.9204808136846971, 50: 0.9232547387887193, 25: 0.9288025889967637, 15: 0.9445214979195562, 10: 0.960702727693019, 5: 0.9727230698104484, 1: 0.9713361072584373, 0: 0.970873786407767}, 'auroc': {100: 0.5264643953882631, 99: 0.42490995529438735, 95: 0.4566111215015294, 90: 0.4345614807753799, 85: 0.4812625564853304, 75: 0.5244078467098237, 50: 0.6631513535363285, 25: 0.9240306242496275, 15: 0.9685280933448366, 10: 0.9822896996096555, 5: 0.9890649226852326, 1: 0.9897074527453831, 0: 0.9897165024645402}, 'preds': {100: array([[0.90076786, 0.09923208],
       [0.90076786, 0.09923208],
       [0.90076786, 0.09923208],
       ...,
       [0.90076786, 0.09923208],
       [0.90076786, 0.09923208],
       [0.90076786, 0.09923208]], dtype=float32), 99: array([[0.90024287, 0.09975714],
       [0.9002466 , 0.09975336],
       [0.90030277, 

/vast/nj594/xai/helpers/evaluate.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  r_ = np.array([[float(p), n] for p,n in r.items()])


In [9]:
for method in ['fastshap-dkl', 'kernelshap_s-dkl']:
    with open('results-{}.pkl'.format(method), 'rb') as f:
        results[method] = pickle.load(f)

In [9]:
# with open('results.pkl', 'wb') as f:
#     pickle.dump(results, f)

In [8]:
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)

In [6]:
results_master['gradcam'] = results['gradcam']

In [11]:
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [10]:
results.keys()

dict_keys(['fastshap', 'fastshap-data', 'kernelshap', 'smoothgrad', 'integratedgradients', 'kernelshap_s', 'kernelshap_s-data', 'realx', 'realx-data', 'gradcam', 'lime', 'fastshap-dkl', 'kernelshap_s-dkl'])